# <font style="color:blue">Project 2: Kaggle Competition - Classification</font>

#### Maximum Points: 100

<div>
    <table>
        <tr><td><h3>Sr. no.</h3></td> <td><h3>Section</h3></td> <td><h3>Points</h3></td> </tr>
        <tr><td><h3>1</h3></td> <td><h3>Data Loader</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>2</h3></td> <td><h3>Configuration</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>3</h3></td> <td><h3>Evaluation Metric</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>4</h3></td> <td><h3>Train and Validation</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>5</h3></td> <td><h3>Model</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>6</h3></td> <td><h3>Utils</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>7</h3></td> <td><h3>Experiment</h3></td><td><h3>5</h3></td> </tr>
        <tr><td><h3>8</h3></td> <td><h3>TensorBoard Dev Scalars Log Link</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>9</h3></td> <td><h3>Kaggle Profile Link</h3></td> <td><h3>50</h3></td> </tr>
    </table>
</div>


In [0]:
import os
import time
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn import metrics
from collections import Counter
from dataclasses import dataclass
from datetime import timedelta

## <font style="color:green">1. Data Loader [10 Points]</font>

In this section, you have to write a class or methods that will be used to get training and validation data loader.

For example,

```
def get_data(args1, *agrs):
    ....
    ....
    return train_loader, test_loader
```

In [0]:
class KenyanFood13Dataset(Dataset):
    """
    This custom dataset class take root directory and train flag, 
    and return dataset training dataset id train flag is true 
    else is return validation dataset.
    """
    
    def __init__(self, data_root, data_set='train', folds=5, fold=0, transform=None):
        
        """
        init method of the class.
        
         Parameters:
         data_root (string): path of root directory.
         data_set (string): 'train' | 'val' | 'test'
         folds (int): number of Folds
         fold (int): part of trainingdata for validation (StratifiedKFold)                        
         transform (method): method that will take PIL image and transforms it.
         
        """
        
        # set transform attribute
        self.transform = transform    #Transforms passed to initalizer
        
        # training data path, this will be used as data root
        self.img_dir = os.path.join(data_root, 'images', 'images')
        
        # Training Data
        train_csv_path = os.path.join(data_root, 'train.csv')       ##loads the images
        train_data = pd.read_csv(train_csv_path, delimiter=',')
        
        # Get all Classes
        self.classes = train_data['class'].unique()
        self.classes.sort()
        
        # number of classes 
        self.num_classes = len(self.classes)
        
        # Label -> Id
        self._label_to_id = {}
        
        # Id -> Label
        self._id_to_label = {}
        
        for i, label in enumerate(self.classes):
            self._label_to_id[label] = i
            self._id_to_label[i] = label
        
        # map class names to clas ids
        train_data['class_id'] = train_data['class'].map(self._label_to_id)
        
        
        if data_set == 'test':
            self.train = False
            
            test_csv_path = os.path.join(data_root, 'test.csv')
            self.data_df= pd.read_csv(test_csv_path, delimiter=',')
             
        else:
            self.train = True
            
            # Use a stratified Split for validation during training
            cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
            split = list(cv.split(train_data, train_data['class_id']))

            self.train_idx, self.val_idx = split[fold]
        
            train_df = train_data.iloc[self.train_idx]
            val_df = train_data.iloc[self.val_idx]
            
           
            # self.counter = collections.Counter(train_df['class_id'].values)
            # self.class_count = np.zeros(self.num_classes)
            # for i in range(self.num_classes):
            #     self.class_count[i] = self.counter[i]
                
            # self.class_weights = [1 - (x / np.sum(self.class_count)) for x in self.class_count] 
            
            self.class_weights = compute_class_weight('balanced',
                                                      np.unique(train_df['class_id'].values),
                                                      train_df['class_id'].values)
            
            if data_set == 'train':
                self.data_df = train_df
            else:
                self.data_df = val_df
            
        
    def __len__(self):
        """
        return length of the dataset
        """
        return self.data_df.shape[0]
    
    
    def __getitem__(self, idx):
        """
        For given index, return images and preprocessing.
        """
        
        img_path = os.path.join(self.img_dir, str(self.data_df['id'].iloc[idx]) + '.jpg')
        
        img = Image.open(img_path).convert("RGB")
        
        if self.transform is not None:
            img = self.transform(img)
            
        if self.train:  
            target = self.data_df['class_id'].iloc[idx]
            return img, target  

        else:
            image_id = self.data_df['id'].iloc[idx]
            return img
                
        
    def label_to_id(self, label):
        """
        latin name mapping to class label id
        """
        return self._label_to_id[label]
    
    def id_to_label(self, id):
        """
        class label to latin name mapping
        """
        return self._id_to_label[id]
    
    def get_split(self):
        """
        get the split ids
        """
        return self.train_idx, self.val_idx

In [0]:
def validation_transforms():
    
    # mean and std of imagenet dataset
    mean = [0.485, 0.456, 0.406] 
    std = [0.229, 0.224, 0.225]

    tf = transforms.Compose([transforms.Resize(320),
                             transforms.CenterCrop(288),
                             transforms.ToTensor(),
                             transforms.Normalize(mean, std)
                             ])
    return tf

In [0]:
def training_transforms():
    
    # mean and std of imagenet dataset
    mean = [0.485, 0.456, 0.406] 
    std = [0.229, 0.224, 0.225]
    
    tf = transforms.Compose([transforms.Resize(320),
                             transforms.ColorJitter(brightness=0.2,
                                                    saturation=0.2,
                                                    contrast=0.2),
                             transforms.RandomAffine(degrees=0.45,
                                                      translate=(0.2, 0.2),
                                                      scale=(0.8, 1.2)),
                             transforms.RandomHorizontalFlip(0.5),
                             transforms.RandomVerticalFlip(0.5),
                             transforms.RandomCrop(288),
                             transforms.ToTensor(),
                             transforms.Normalize(mean, std),
                             ])                                
    return tf

In [0]:
def get_data(batch_size, data_root, num_workers=4, folds=5, fold=0):
    
    # train
    train_dataset =  KenyanFood13Dataset(data_root, data_set='train',
                                         folds=folds,
                                         fold=fold,
                                         transform=training_transforms())
    
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True,
                                               num_workers=num_workers)
    # validation
    val_dataset = KenyanFood13Dataset(data_root,
                                      data_set='val',
                                      folds=folds,
                                      fold=fold,
                                      transform=validation_transforms())
    
    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             shuffle=False,
                                             num_workers=num_workers)
    
    # test
    test_dataset = KenyanFood13Dataset(data_root,
                                       data_set='test',
                                       transform=validation_transforms())
    
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=batch_size,
                                              shuffle=False,
                                              num_workers=num_workers)
    
    
    return train_loader, val_loader, test_loader

## <font style="color:green">2. Configuration [5 Points]</font>

Define your configuration in this section.

For example,

```
@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    batch_size: int = 10 
    epochs_count: int = 50  
    init_learning_rate: float = 0.1  # initial learning rate for lr scheduler
    log_interval: int = 5  
    test_interval: int = 1  
    data_root: str = "./cat-dog-panda" 
    num_workers: int = 2  
    device: str = 'cuda'  
    
```

In [0]:
@dataclass
class SystemConfiguration:
    '''
    Describes the common system setting needed for reproducible training
    '''
    seed: int = 21  # seed number to set the state of all random number generators
    cudnn_benchmark_enabled: bool = True  # enable CuDNN benchmark for the sake of performance
    cudnn_deterministic: bool = True  # make cudnn deterministic (reproducible training)

In [0]:
@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    batch_size: int = 15
    epochs_count: int = 20 
    init_learning_rate: float = 0.0005
    decay_rate: float = 0.2  
    log_interval: int = 500  
    test_interval: int = 1  
    data_root: str = "./" 
    folds: int = 5   # split dataset in number of folds
    fold: int = 0    # current fold for training
    num_workers: int = 2 
 #   model: str = 'resnext50_32x4d'
    model: str = 'inception_v3'

    
#    save_name: str = "resnext50_32x4d.pt"
    save_name: str = "inception_v31.pt"

    device: str = 'cuda'

In [0]:
def setup_system(system_config: SystemConfiguration) -> None:
    torch.manual_seed(system_config.seed)
    if torch.cuda.is_available():
        torch.backends.cudnn_benchmark_enabled = system_config.cudnn_benchmark_enabled
        torch.backends.cudnn.deterministic = system_config.cudnn_deterministic

## <font style="color:green">3. Evaluation Metric [10 Points]</font>

Define methods or classes that will be used in model evaluation, for example, accuracy, f1-score, etc.

In [0]:
def accuracy_torch(y_true:torch.Tensor, y_pred:torch.Tensor) -> torch.Tensor:
    
    '''Calculate Accuracy.'''
    
    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)
    
    accuracy = y_pred.eq(y_true).float().mean()
    
    return accuracy

In [0]:
def accuracy_numpy(y_true, y_pred):
    
    '''Calculate Accuracy.'''
    
    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)
    
    accuracy = np.equal(y_pred, y_true).astype(np.int).mean()
    
    return accuracy

In [0]:
def confusion_matrix(y_true, y_pred):
    
    '''Calculate Confusion Matrix'''
    
    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)
    
    return metrics.confusion_matrix(y_true, y_pred)

In [0]:
def classification_report(y_true, y_pred, target_names=None):
    
    '''Classification Report:
       per class:  precision
                   recall
                   f-score
    '''
    
    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)
    
    report = metrics.classification_report(y_true,
                                           y_pred,
                                           target_names=target_names,
                                           digits=3,
                                           zero_division=0)
    
    return report
    

## <font style="color:green">4. Train and Validation [5 Points]</font>

Write the methods or classes that will be used for training and validation.

In [0]:
def train(train_config: TrainingConfiguration, model: nn.Module, optimizer: torch.optim.Optimizer,
          train_loader: torch.utils.data.DataLoader, epoch_idx: int, tb_writer: SummaryWriter) -> None:
    
    # change model in training mood
    model.train()
    
    # to get batch loss
    batch_loss = np.array([])
    
    # to get batch accuracy
    batch_acc = np.array([])
    
    # class weights
    class_weights = train_loader.dataset.class_weights
    #print("Using Class Weights:")
    #for i, weight in enumerate(class_weights):
    #    print("{} : {}".format(i, weight))
    class_weights = torch.FloatTensor(class_weights).to(TrainingConfiguration.device)
    weighted_cross_entropy = nn.CrossEntropyLoss(weight=class_weights)
        
    for batch_idx, (data, target) in enumerate(train_loader):
        
        # clone target
        indx_target = target.clone()
        # send data to device (its is mandatory if GPU has to be used)
        data = data.to(train_config.device)
        # send target to device
        target = target.to(train_config.device)

        # reset parameters gradient to zero
        optimizer.zero_grad()
        
        # forward pass to the model
        output = model(data)
        
        # cross entropy loss
        loss = weighted_cross_entropy(output, target)
        #loss = F.cross_entropy(output, target)
        
        # find gradients w.r.t training parameters
        loss.backward()
        # Update parameters using gardients
        optimizer.step()
        
        batch_loss = np.append(batch_loss, [loss.item()])
        
        # Score to probability using softmax
        prob = F.softmax(output, dim=1)
            
        # get the index of the max probability
        pred = prob.data.max(dim=1)[1]  
                        
        # correct prediction
        correct = pred.cpu().eq(indx_target).sum()
            
        # accuracy
        acc = float(correct) / float(len(data))
        
        batch_acc = np.append(batch_acc, [acc])

        if batch_idx % train_config.log_interval == 0 and batch_idx > 0:
            
            total_batch = epoch_idx * len(train_loader.dataset)/train_config.batch_size + batch_idx
            tb_writer.add_scalar('Loss/train-batch', loss.item(), total_batch)
            tb_writer.add_scalar('Accuracy/train-batch', acc, total_batch)
            
    epoch_loss = batch_loss.mean()
    epoch_acc = batch_acc.mean()
    print('Train Loss: {:.6f} Accuracy: {:.4f}'.format(epoch_loss, epoch_acc))
    return epoch_loss, epoch_acc

In [0]:
def validate_batch(model, device, batch_input):
    """
    get prediction for batch inputs
    """
    
    # send model to cpu/cuda according to your system configuration
    model.to(device)
    
    # it is important to do model.eval() before prediction
    model.eval()
    
    data, target = batch_input

    data = data.to(device)
    target = target.to(device)

    output = model(data)
    
    # Loss
    loss = F.cross_entropy(output, target).item()
    
    # Score to probability using softmax
    prob = F.softmax(output, dim=1)
    
    return loss, prob.data.cpu().numpy()

In [0]:
def validate_dataset(model, dataloader, device):
    """
    get targets and prediction probabilities
    """
    
    losses = []
    probs = []
    targets = []
    
    for _, (data, target) in enumerate(dataloader):
        
        loss, prob = validate_batch(model, device, (data, target))
        
        losses.append(loss)
        probs.append(prob)
        targets.append(target.numpy())
        

        
    targets = np.concatenate(targets)
    targets = targets.astype(np.int)
    
    probs = np.concatenate(probs, axis=0)
    
    losses = np.array(loss)
    
    return losses, targets, probs

In [0]:
def validate(train_config: TrainingConfiguration,
             model: nn.Module,
             dataloader: torch.utils.data.DataLoader) -> float:
    
    
    losses, targets, probs = validate_dataset(model, dataloader, train_config.device)
    
    loss = losses.mean()
    
    predictions = probs.argmax(axis=1)
    count_corect_predictions = np.equal(targets, predictions).astype(np.int).sum()
    
    # calculate metric
    accuracy = accuracy_numpy(targets, predictions)
    
    print('Validation loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)'.format(
          loss, count_corect_predictions, len(dataloader.dataset), accuracy * 100))
    
    report = classification_report(targets, predictions, target_names=dataloader.dataset.classes)
    
    return loss, accuracy, report

In [0]:
def predict(train_config: TrainingConfiguration,
            model: nn.Module,
            dataloader: torch.utils.data.DataLoader) -> float:
    
    probs = []
    model.to(TrainingConfiguration.device)
    model.eval()
    
    for data in dataloader:
        
        data = data.to(TrainingConfiguration.device)
        
        output = model(data)
        
        # Score to probability using softmax
        prob = F.softmax(output, dim=1)
        
        probs.append(prob.cpu().detach().numpy())
       
    probs = np.concatenate(probs, axis=0)
    
    return probs

## <font style="color:green">5. Model [5 Points]</font>

Define your model in this section.

In [0]:
def inception_v3(feature_extract= True, num_class=13):
  model =models.inception_v3(pretrained=True,aux_logits=False)

  if feature_extract:
    for param in model.parameters():
      param.requires_grad = False


    # for param in model.Mixed_6a.parameters():
      
    #   param.requires_grad=True



    # for param in model.Mixed_6b.parameters():
      
    #   param.requires_grad=True




    # for param in model.Mixed_6c.parameters():
      
    #   param.requires_grad=True


    # for param in model.Mixed_6d.parameters():
      
    #   param.requires_grad=True



    # for param in model.Mixed_6e.parameters():
      
    #   param.requires_grad=True


    # for param in model.Mixed_7a.parameters():
      
    #   param.requires_grad=True

    # for param in model.Mixed_7b.parameters():
      
    #   param.requires_grad=True


    # for param in model.Mixed_7c.parameters():
      
    #   param.requires_grad=True
      


    # for param in model.parameters():
    #   param.aux_logits = False




    last_layer_in = model.fc.in_features

    model.fc = nn.Linear(last_layer_in , num_class)


  return model


In [0]:
def resnext50_32x4d(transfer_learning=True, num_class=13):
    model = models.resnext50_32x4d(pretrained=True)
    
    if transfer_learning:
        for param in model.parameters():
            param.requires_grad = False

        # for param in model.layer3.parameters():
          
          
        #   param.requires_grad = True
            
        # for param in model.layer4.parameters():
        #     param.requires_grad = True
                              
    last_layer_in = model.fc.in_features
    model.fc = nn.Linear(last_layer_in, num_class)
    
    return model

In [0]:
def resnet50(transfer_learning=True, num_class=13):
    model = models.resnet50(pretrained=True)
    
    if transfer_learning:
        for param in model.parameters():
            param.requires_grad = False

        
        for param in model.layer3.parameters():
            param.requires_grad = True
            
        for param in model.layer4.parameters():
            param.requires_grad = True
                   
    last_layer_in = model.fc.in_features
    model.fc = nn.Linear(last_layer_in, num_class)
    
    return model

## <font style="color:green">6. Utils [5 Points]</font>

Define your methods or classes which are not covered in the above sections.

In [0]:
def save_model(model, device, model_dir='models', model_file_name='classifier.pt'):
    

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    model_path = os.path.join(model_dir, model_file_name)

    # make sure you transfer the model to cpu.
    if device == 'cuda':
        model.to('cpu')

    # save the state_dict
    torch.save(model.state_dict(), model_path)
    
    if device == 'cuda':
        model.to('cuda')
    
    return

In [0]:
def load_model(model, model_dir='models', model_file_name='cat_dog_panda_classifier.pt'):
    model_path = os.path.join(model_dir, model_file_name)

    # loading the model and getting model parameters by using load_state_dict
    model.load_state_dict(torch.load(model_path))
    
    return model

In [0]:
def add_pr_curves_to_tensorboard(model, dataloader, device, tb_writer, epoch):
    """
    Add precession and recall curve to tensorboard.
    """
    
    _, targets, pred_prob = validate_dataset(model, dataloader, device)
    
    for cls_idx in range(dataloader.dataset.num_classes):
        binary_target = targets == cls_idx
        true_prediction_prob = pred_prob[:, cls_idx]
        
        tb_writer.add_pr_curve(dataloader.dataset.id_to_label(cls_idx), 
                               binary_target, 
                               true_prediction_prob, 
                               global_step=epoch)
        
    return

In [0]:
def add_model_weights_as_histogram(model, tb_writer, epoch):
    for name, param in model.named_parameters():
        tb_writer.add_histogram(name.replace('.', '/'), param.data.cpu().abs(), epoch)
    return

In [0]:
def add_network_graph_tensorboard(model, inputs, tb_writer):
    tb_writer.add_graph(model, inputs)
    return

In [0]:
def sec_to_min(seconds):
    
    seconds = int(seconds)
    minutes = seconds // 60
    seconds_remaining = seconds % 60
    
    if seconds_remaining < 10:
        seconds_remaining = '0{}'.format(seconds_remaining)
    
    return '{}:{}'.format(minutes, seconds_remaining)

In [0]:
def sec_to_time(seconds):
    return "{:0>8}".format(str(timedelta(seconds=int(seconds))))

In [0]:
def print_seperator(length=80, character='-', name=None):
    
    if name:
        rest = len(name) % 2
        repeat = length // 2 - len(name) // 2 - 2
        seperator_1 = character * repeat
        
        if rest:
            seperator_2 = character * (repeat-1)
        else:
            seperator_2 = seperator_1
    
        print("{}[ {} ]{}".format(seperator_1, name, seperator_2))
         
    else:
        seperator = character * length
        print(seperator)

## <font style="color:green">7. Experiment [5 Points]</font>

Choose your optimizer and LR-scheduler and use the above methods and classes to train your model.

#### Select Model:

In [0]:
# intialize config
train_config=TrainingConfiguration()
sys_config = SystemConfiguration()

# set Model
train_config.model = 'inception_v3'
#train_config.model = 'resnet50'

#### Optimizer and Scheduler:

In [0]:
def get_optimizer_and_scheduler(model):
    train_config = TrainingConfiguration()

    init_learning_rate = train_config.init_learning_rate

    # optimizer
    optimizer = optim.Adam(model.parameters(),
                           lr = init_learning_rate)

    decay_rate = train_config.decay_rate

    lmbda = lambda epoch: 1/(1 + decay_rate * epoch)

    # Scheduler
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lmbda)
    
    return optimizer, scheduler

#### Main Function:

In [0]:
def main(model, optimizer, tb_writer, scheduler=None, system_configuration=SystemConfiguration(), 
         training_configuration=TrainingConfiguration()):
    
    # system configuration
    setup_system(system_configuration)

    # batch size
    batch_size_to_set = training_configuration.batch_size
    
    # num_workers
    num_workers_to_set = training_configuration.num_workers
    
    # if GPU is available use training config, 
    # else lowers batch_size, num_workers and epochs count
    if torch.cuda.is_available():
        training_configuration.device = "cuda"
    else:
        training_configuration.device = "cpu"
        training_configuration.batch_size_to_set = 16
        training_configuration.num_workers_to_set = 2

    # data loader
    train_loader, test_loader, _ = get_data(batch_size=batch_size_to_set,
                                            data_root=training_configuration.data_root,
                                            num_workers=num_workers_to_set,
                                            folds=training_configuration.folds,
                                            fold=training_configuration.fold)
      
    # send model to device (GPU/CPU)
    model.to(training_configuration.device)
    
    
    # add network graph with inputs info
    images, labels = next(iter(test_loader))
    images = images.to(training_configuration.device)
    add_network_graph_tensorboard(model, images, tb_writer)

    best_loss = torch.tensor(np.inf)
    best_accuracy = 0
    best_report = None
    best_epoch = 0
    
    # epoch train/test loss
    epoch_train_loss = np.array([])
    epoch_test_loss = np.array([])
    
    # epch train/test accuracy
    epoch_train_acc = np.array([])
    epoch_test_acc = np.array([])  
    
    # Start Training
    print_seperator(80, '-', name='Training: {}'.format(training_configuration.model))
    print("Batch Size: {}".format(training_configuration.batch_size))
    print("Epochs: {}".format(training_configuration.epochs_count))
    print("Fold: {} ({}/{})".format(training_configuration.fold,
                                    training_configuration.fold+1,
                                    training_configuration.folds))
    
    
    
    # Calculate Initial Test Loss
    print_seperator(80, '-', name='Initial Fold: {}'.format(training_configuration.fold))
    init_val_loss, init_val_accuracy, _ = validate(training_configuration, model, test_loader)
    print("Initial Validation Loss : {:.6f}, \nInitial Validation Accuracy : {:.3f}%".format(init_val_loss, init_val_accuracy*100))
    

    # trainig time measurement
    t_begin = time.time()
    for epoch in range(training_configuration.epochs_count):
        
        print_seperator(80, '-', name='Epoch: {}'.format(epoch))
        if scheduler is not None:
            tb_writer.add_scalar('Learning_Rate/schedule', scheduler.get_last_lr()[0], epoch)
        else:
            tb_writer.add_scalar('Learning_Rate/schedule', training_configuration.init_learning_rate, epoch)    
            
        # Train
        train_loss, train_acc = train(training_configuration, model, optimizer, train_loader, epoch, tb_writer)
        
        epoch_train_loss = np.append(epoch_train_loss, [train_loss])
        
        epoch_train_acc = np.append(epoch_train_acc, [train_acc])
        
        # add scalar (loss/accuracy) to tensorboard
        tb_writer.add_scalar('Loss/Train',train_loss, epoch)
        tb_writer.add_scalar('Accuracy/Train', train_acc, epoch)

        elapsed_time = time.time() - t_begin
        speed_epoch = elapsed_time / (epoch + 1)
        speed_batch = speed_epoch / len(train_loader)
        eta = speed_epoch * training_configuration.epochs_count - elapsed_time
        
        print("Elapsed {}, {} time/epoch, {:.2f} s/batch, remaining {}".format(
                sec_to_time(elapsed_time), sec_to_time(speed_epoch), speed_batch, sec_to_time(eta)))
        
        # add time metadata to tensorboard
        tb_writer.add_scalar('Time/min_training', elapsed_time/60, epoch)
        tb_writer.add_scalar('Time/min_per_epoch', speed_epoch/60, epoch)
        tb_writer.add_scalar('Time/sec_per_batch', speed_batch, epoch)
        tb_writer.add_scalar('Time/min_remaining', eta/60, epoch)
        

        # Validate
        if epoch % training_configuration.test_interval == 0:
            current_loss, current_accuracy, report = validate(training_configuration, model, test_loader)
            
            epoch_test_loss = np.append(epoch_test_loss, [current_loss])
        
            epoch_test_acc = np.append(epoch_test_acc, [current_accuracy])
            
            
            if current_accuracy > best_accuracy:
                best_accuracy = current_accuracy
                best_report = report
                best_epoch = epoch
                print('Model Improved. Saving the Model.')
                save_model(model, 
                           device=training_configuration.device,
                           model_file_name=training_configuration.save_name)
            
            
            
            # add scalar (loss/accuracy) to tensorboard
            tb_writer.add_scalar('Loss/Validation', current_loss, epoch)
            tb_writer.add_scalar('Accuracy/Validation', current_accuracy, epoch)
            
            # add scalars (loss/accuracy) to tensorboard
            #tb_writer.add_scalars('Loss/train-val', {'train': train_loss, 
            #                               'validation': current_loss}, epoch)
            #tb_writer.add_scalars('Accuracy/train-val', {'train': train_acc, 
            #                                   'validation': current_accuracy}, epoch)
            
            if current_loss < best_loss:
                best_loss = current_loss
                        
        # scheduler step/ update learning rate
        if scheduler is not None:
            scheduler.step()
              
        # adding model weights to tensorboard as histogram
        #add_model_weights_as_histogram(model, tb_writer, epoch)
        
        # add pr curves to tensor board
        #add_pr_curves_to_tensorboard(model, test_loader, 
        #                             training_configuration.device, 
        #                             tb_writer, epoch)
        
    print_seperator(80, '-', name='Result Fold: {}'.format(training_configuration.fold))
    print("Total time: {} - Accuracy: {:.3f}% - Best Epoch: {}".format(sec_to_time(time.time()-t_begin),
                                                                          best_accuracy * 100,
                                                                          best_epoch))
    print(best_report)
    print_seperator(80, '-', name='End Fold: {}'.format(training_configuration.fold))
    print()
    
    
    return model, epoch_train_loss, epoch_train_acc, epoch_test_loss, epoch_test_acc

#### Train Folds:

In [0]:
# cllect results of all folds
train_results_folds = {}

# train folds for crossvalidation
for fold in range(0, train_config.folds):
    
    if train_config.model == 'inception_v3':
         #model = torch.hub.load('pytorch/vision:v0.5.0', 'inception_v3', pretrained=True,,num_class=13,aux_logits=False)

         #model = resnext50_32x4d(transfer_learning=True, num_class=13)
         model = inception_v3(feature_extract=True, num_class=13)
    else:
      
        model = resnet50(transfer_learning=True, num_class=13)

    # get optimizer and scheduler
    optimizer, scheduler = get_optimizer_and_scheduler(model)
        
    # tensorboard summary writer
    tb_writer = SummaryWriter('logs/fold_{}/{}'.format(fold, train_config.model))  
    
    # set training config current fold and save name
    train_config.fold = fold
    train_config.save_name = 'fold_{}_{}.pt'.format(fold, train_config.model)
    
    # train and validate
    model, train_loss, train_acc, val_loss, val_acc = main(model, 
                                                           optimizer,
                                                           tb_writer,
                                                           scheduler,
                                                           sys_config,
                                                           training_configuration=train_config)
    
    # collect results of all folds of current fold
    train_results_folds[fold] = {"train_loss": train_loss,
                                 "train_acc": train_acc,
                                 "val_loss": val_loss,
                                 "val_acc": val_acc}
    
    # save results dictionary
    with open('./logs/train_results_{}.pk'.format(train_config.model), 'wb') as file:
        pickle.dump(train_results_folds, file, protocol=pickle.HIGHEST_PROTOCOL)
    
    tb_writer.close()

---------------------------[ Training: inception_v3 ]---------------------------
Batch Size: 15
Epochs: 20
Fold: 0 (1/5)
-------------------------------[ Initial Fold: 0 ]------------------------------
Validation loss: 2.5258, Accuracy: 88/1308 (6.728%)
Initial Validation Loss : 2.525790, 
Initial Validation Accuracy : 6.728%
----------------------------------[ Epoch: 0 ]----------------------------------
Train Loss: 2.339771 Accuracy: 0.2607
Elapsed 00:02:15, 00:02:15 time/epoch, 0.39 s/batch, remaining 00:42:53
Validation loss: 1.8778, Accuracy: 557/1308 (42.584%)
Model Improved. Saving the Model.
----------------------------------[ Epoch: 1 ]----------------------------------
Train Loss: 2.015054 Accuracy: 0.3927
Elapsed 00:04:59, 00:02:29 time/epoch, 0.43 s/batch, remaining 00:44:55
Validation loss: 1.5363, Accuracy: 635/1308 (48.547%)
Model Improved. Saving the Model.
----------------------------------[ Epoch: 2 ]----------------------------------
Train Loss: 1.909825 Accuracy: 0.

#### Cross-Validation Result:

In [0]:
# Calculate Crossvalidation Result
best_acc_folds = np.zeros(train_config.folds, dtype=np.float32)  

print_seperator(80, '-', name='Cross-Validation Accuracy')

for i in range(train_config.folds):
    acc_fold_epochs = train_results_folds[i]["val_acc"]
    
    best_acc_fold = acc_fold_epochs.max()
    best_acc_epoch = np.argmax(acc_fold_epochs)
    
    print("Fold_{}: {:.3f} (Epoch:{})".format(i, best_acc_fold, best_acc_epoch))
    
    best_acc_folds[i]= best_acc_fold
    
crossvalidation_accuracy = best_acc_folds.mean()


print("\nCross-Validation Accuracy: {:.3f}%".format(crossvalidation_accuracy*100))
print_seperator(80, '-', name='End Cross-Validation')

### <font style="color:green">7.1 Predict and submission.csv</font>

In [0]:
probs_folds = []

if train_config.model == 'resnext50_32x4d':
    model = resnext50_32x4d(transfer_learning=True, num_class=13)
else:
    model = resnet50(transfer_learning=True, num_class=13)
    
for param in model.parameters():
    param.requires_grad = False

for fold in range(0, train_config.folds):
    print("Predict Fold: {}".format(fold))
    train_config.fold = fold
    train_config.save_name = 'fold_{}_{}.pt'.format(fold, train_config.model)
    
    model = load_model(model, model_dir='models', model_file_name=train_config.save_name)
    
    # data loader
    _, _, test_loader = get_data(batch_size=train_config.batch_size,
                                 data_root=train_config.data_root,
                                 num_workers=train_config.num_workers,
                                 folds=train_config.folds,
                                 fold=train_config.fold)
    
    probs = predict(train_config, model, test_loader)
    
    probs_folds.append(probs)

In [0]:
# Sum probs over all folds
probs = np.array(probs_folds).sum(0)
predictions = np.argmax(probs,axis=1)

In [0]:
# create submmison dataframe
submission_df = test_loader.dataset.data_df
submission_df["class"] = predictions

submission_df["class"] = submission_df["class"].map(test_loader.dataset.id_to_label)

submission_df.head(10)

In [0]:
# Save submission dataframe
submission_path = os.path.join(train_config.data_root, 'submission.csv')
submission_df.to_csv(submission_path, index=False)

## <font style="color:green">8. TensorBoard Dev Scalars Log Link [5 Points]</font>

Share your tensorboard scalars logs link in this section. You can also share (not mandatory) your GitHub link if you have pushed this project in GitHub. 

For example, [Find Project2 logs here](https://tensorboard.dev/experiment/kMJ4YU0wSNG0IkjrluQ5Dg/#scalars).

Link: https://tensorboard.dev/experiment/iDofl6ypRDS3wdCZBrlkQQ/

## <font style="color:green">9. Kaggle Profile Link [50 Points]</font>

Share your Kaggle profile link here with us so that we can give points for the competition score. 

Kaggle Link: https://www.kaggle.com/khabel